In [1]:
from azure.cosmos import CosmosClient, exceptions, PartitionKey
import os
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [2]:
# Set the Cosmos DB endpoint, key and database name in the .env file. The key and endpoint can be found in the resource created in the portal.
COSMOS_ENDPOINT = os.environ["COSMOS_ENDPOINT"]
COSMOS_KEY =  os.environ["COSMOS_KEY"]
client = CosmosClient(COSMOS_ENDPOINT, credential=COSMOS_KEY)
DATABASE_NAME = 'azqr-database'
CONTAINER_NAME = 'customers'

In [3]:
# Create the database if it doesnt already exist
client = CosmosClient(COSMOS_ENDPOINT, credential=COSMOS_KEY)
try:
    database = client.create_database(DATABASE_NAME)
except exceptions.CosmosResourceExistsError:
    database = client.get_database_client(DATABASE_NAME)

print(database)

<DatabaseProxy [dbs/azqr-database]>


In [4]:
# Create the container if it doesnt already exist
try:
    container = database.create_container(id=CONTAINER_NAME, partition_key=PartitionKey(path="/id"))
except exceptions.CosmosResourceExistsError:
    container = database.get_container_client(CONTAINER_NAME)
except exceptions.CosmosHttpResponseError:
    raise
print(container)

<ContainerProxy [dbs/azqr-database/colls/customers]>


In [13]:
# Loop through each json file in data/customer_info and insert into container
import os
import json
import glob
path = '../data/cosmosdb/upload'
for filename in glob.glob(os.path.join(path, '*.json')):
    print('Processing file {0}'.format(filename))
    with open(filename) as file:
        customer = json.load(file)
        print('Upserting item: ', customer)
        container.upsert_item(customer)
        print('Upserted item with id {0}'.format(customer['id']))

Processing file ../data/cosmosdb/upload/customer_info_1.json
Upserting item:  {'id': '1', 'name': 'DataExperts', 'subscription': 'a0e36c1b-7b3e-49ff-b98c-2e75c456d1a2', 'tenant': 'a0e36c1b-7b3e-49ff-b98c-2e75c456d1a2', 'advisor': [{'Subscription': 'a0e36c1b-7b3e-49ff-b98c-2e75c456d1a2', 'Name': '8d2c636f-918d-4ef6-8454-8f5f4b086e72', 'Type': 'Microsoft.Subscriptions/subscriptions', 'Category': 'HighAvailability', 'Description': 'Create an Azure Service Health alert', 'PotentialBenefits': '', 'Risk': '', 'LearnMoreLink': ''}, {'Subscription': 'a0e36c1b-7b3e-49ff-b98c-2e75c456d1a2', 'Name': 'Maya01-vm01_OsDisk_1_288aebbc91334375a77a78ec91ea8c1d', 'Type': 'MICROSOFT.COMPUTE/DISKS', 'Category': 'HighAvailability', 'Description': 'Use Azure Disks with Zone Redundant Storage for higher resiliency and availability', 'PotentialBenefits': '', 'Risk': '', 'LearnMoreLink': ''}, {'Subscription': 'a0e36c1b-7b3e-49ff-b98c-2e75c456d1a2', 'Name': 'SecLab-VM01_OsDisk_1_88c2f1cd4c8849ababac7a9a3b2583df'

In [5]:
# Get items from container to validate they were inserted
print('Get all items in container')
items = list(container.read_all_items(max_item_count=10))
print(items)


Get all items in container
[{'id': '1', 'name': 'DataExperts', 'subscription': 'a0e36c1b-7b3e-49ff-b98c-2e75c456d1a2', 'tenant': 'a0e36c1b-7b3e-49ff-b98c-2e75c456d1a2', 'advisor': [{'Subscription': 'a0e36c1b-7b3e-49ff-b98c-2e75c456d1a2', 'Name': '8d2c636f-918d-4ef6-8454-8f5f4b086e72', 'Type': 'Microsoft.Subscriptions/subscriptions', 'Category': 'HighAvailability', 'Description': 'Create an Azure Service Health alert', 'PotentialBenefits': '', 'Risk': '', 'LearnMoreLink': ''}, {'Subscription': 'a0e36c1b-7b3e-49ff-b98c-2e75c456d1a2', 'Name': 'Maya01-vm01_OsDisk_1_288aebbc91334375a77a78ec91ea8c1d', 'Type': 'MICROSOFT.COMPUTE/DISKS', 'Category': 'HighAvailability', 'Description': 'Use Azure Disks with Zone Redundant Storage for higher resiliency and availability', 'PotentialBenefits': '', 'Risk': '', 'LearnMoreLink': ''}, {'Subscription': 'a0e36c1b-7b3e-49ff-b98c-2e75c456d1a2', 'Name': 'SecLab-VM01_OsDisk_1_88c2f1cd4c8849ababac7a9a3b2583df', 'Type': 'MICROSOFT.COMPUTE/DISKS', 'Category': '